In [3]:
%%bash
#!/usr/bin/env bash
set -euo pipefail

mkdir -p week5_data refs align vcf phase igv tmp isec

: "${THREADS:=4}"

have() { command -v "$1" >/dev/null 2>&1; }

echo "minimap2: $(minimap2 --version)"
echo "samtools: $(samtools --version | head -n1)"
echo "bcftools: $(bcftools --version | head -n1)"

if have extractHAIRS; then
  echo "extractHAIRS: OK"
else
  echo "extractHAIRS: MISSING (needed later for phasing)"
fi

if have HAPCUT2; then
  echo "HAPCUT2: OK"
else
  echo "HAPCUT2: MISSING (needed later for phasing)"
fi

if have reformat.sh; then
  echo "BBMap reformat.sh: OK"
else
  echo "BBMap reformat.sh: MISSING (used to split interleaved Illumina)"
fi

if have java; then
  java -version 2>&1 | head -n1
else
  echo "Java: MISSING (only needed if we use fgbio for VCF conversion fallback)"
fi

minimap2: 2.30-r1287
samtools: samtools 1.22.1
bcftools: bcftools 1.22
extractHAIRS: OK
HAPCUT2: OK
BBMap reformat.sh: MISSING (used to split interleaved Illumina)
Java: MISSING (only needed if we use fgbio for VCF conversion fallback)


Defining gene BED (hg38, +2kb padding)

In [20]:
%%bash
cat > week5_data/cyp2c.hg38.bed <<'BED'
chr10\t94760662\t94858282\tCYP2C19
chr10\t94936658\t94991091\tCYP2C9
chr10\t95034773\t95088997\tCYP2C8
BED

Fetching chr10 reference FASTA (UCSC) + index + minimap2 index

In [7]:
%%bash

set -euo pipefail
REF=refs/chr10.fa.gz
if [ ! -s "$REF" ]; then
  curl -L -o "$REF" "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr10.fa.gz"
fi
# bgzip is not required here; UCSC is already gz
gunzip -c "$REF" > refs/chr10.fa
samtools faidx refs/chr10.fa
minimap2 -d refs/chr10.mmi refs/chr10.fa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.1M  100 41.1M    0     0  43.1M      0 --:--:-- --:--:-- --:--:-- 43.10375kM
[M::mm_idx_gen::4.693*0.76] collected minimizers
[M::mm_idx_gen::5.157*0.97] sorted minimizers
[M::main::6.293*0.98] loaded/built the index for 1 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::6.417*0.98] distinct minimizers: 16061920 (79.91% are singletons); average occurrences: 1.563; average spacing: 5.329; total length: 133797422
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -d refs/chr10.mmi refs/chr10.fa
[M::main] Real time: 6.464 sec; CPU: 6.365 sec; Peak RSS: 1.097 GB


Bringing reads into place

In [11]:
%%bash
set -euo pipefail

# We are already in week5/
mkdir -p tmp

# Illumina (interleaved) — prefer your local file
if [ -s illumina.fq.bz2 ]; then
  ln -sf "$(pwd)/illumina.fq.bz2" tmp/illumina.fq.bz2 || cp -f illumina.fq.bz2 tmp/illumina.fq.bz2
else
  curl -L -o tmp/illumina.fq.bz2 "https://raw.githubusercontent.com/quinlan-lab/datasets/master/fastq/mini_pe_interleaved.fq.bz2"
fi

# PacBio — prefer your local file
if [ -s pacbio.fq.bz2 ]; then
  ln -sf "$(pwd)/pacbio.fq.bz2" tmp/pacbio.fq.bz2 || cp -f pacbio.fq.bz2 tmp/pacbio.fq.bz2
else
  curl -L -o tmp/pacbio.fq.bz2 "https://raw.githubusercontent.com/biocore-ntnu/pb-demo-data/main/mini_pacbio_like.fq.bz2"
fi

echo "tmp/ contents:"
ls -lh tmp

tmp/ contents:
total 0
lrwxrwxrwx 1 biouser biouser 53 Oct 28 16:40 illumina.fq.bz2 -> /home/biouser/fall25-csc-bioinf/week5/illumina.fq.bz2
lrwxrwxrwx 1 biouser biouser 51 Oct 28 16:40 pacbio.fq.bz2 -> /home/biouser/fall25-csc-bioinf/week5/pacbio.fq.bz2


Decompress + (for Illumina) split interleaved -> R1/R2

In [12]:
%%bash
set -euo pipefail

# Decompress (keeps originals)
bunzip2 -fk tmp/illumina.fq.bz2
bunzip2 -fk tmp/pacbio.fq.bz2

# Split interleaved Illumina: 8-line blocks → first 4 lines R1, next 4 lines R2
awk '
  BEGIN{ r1="tmp/illumina_R1.fq"; r2="tmp/illumina_R2.fq" }
  { blk = (NR-1) % 8; if (blk < 4) print >> r1; else print >> r2 }
' tmp/illumina.fq

echo "R1 reads: $(($(wc -l < tmp/illumina_R1.fq)/4))"
echo "R2 reads: $(($(wc -l < tmp/illumina_R2.fq)/4))"

R1 reads: 154753
R2 reads: 154752


Aligning with minimap2 -> sorted, indexed BAMs

In [14]:
%%bash
set -euo pipefail

# Define a default for this cell (each %%bash is a new shell)
THREADS=${THREADS:-4}

R1="tmp/illumina_R1.fq"
R2="tmp/illumina_R2.fq"
PB="tmp/pacbio.fq"
REFIDX="refs/chr10.mmi"

# Sanity checks
for f in "$R1" "$R2" "$PB" "$REFIDX"; do
  if [ ! -s "$f" ]; then
    echo "Missing required file: $f" >&2
    exit 1
  fi
done

# Align Illumina paired-end
minimap2 -t "$THREADS" -ax sr "$REFIDX" "$R1" "$R2" \
  | samtools sort -@ "$THREADS" -o align/illumina.chr10.bam
samtools index align/illumina.chr10.bam

# Align PacBio (switch to map-hifi if these are HiFi/CCS reads)
PRESET="map-pb"
minimap2 -t "$THREADS" -ax "$PRESET" "$REFIDX" "$PB" \
  | samtools sort -@ "$THREADS" -o align/pacbio.chr10.bam
samtools index align/pacbio.chr10.bam

# Quick peek so we know mapping happened
echo "Illumina BAM:"
samtools flagstat align/illumina.chr10.bam | head -n 5
echo
echo "PacBio BAM:"
samtools flagstat align/pacbio.chr10.bam | head -n 5

[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::1.549*1.05] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::1.549*1.05] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::1.668*1.05] distinct minimizers: 16061920 (79.91% are singletons); average occurrences: 1.563; average spacing: 5.329; total length: 133797422
[W::mm_bseq_read_frag2] query files have different number of records; extra records skipped.
[M::worker_pipeline::11.804*3.02] mapped 309504 sequences
[M::main] Version: 2.30-r1287
[M::main] CMD: minimap2 -t 4 -ax sr refs/chr10.mmi tmp/illumina_R1.fq tmp/illumina_R2.fq
[M::main] Real time: 11.820 sec; CPU: 35.615 sec; Peak RSS: 0.944 GB
[bam_sort_core] merging from 0 files and 4 in-memory blocks...
[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::0.930*1.05] loaded/built the index for 1 target sequ

Illumina BAM:
309647 + 0 in total (QC-passed reads + QC-failed reads)
309504 + 0 primary
0 + 0 secondary
143 + 0 supplementary
0 + 0 duplicates

PacBio BAM:
3126 + 0 in total (QC-passed reads + QC-failed reads)
3063 + 0 primary
53 + 0 secondary
10 + 0 supplementary
0 + 0 duplicates


In [ ]:
%%bash
# Fixing BED file formatting issues

set -euo pipefail

# Recreate file using spaces, then canonicalize to tabs (so it's robust if you paste in a Windows terminal)
cat > week5_data/cyp2c.hg38.bed <<'BED'
chr10    94760662    94858282    CYP2C19
chr10    94936658    94991091    CYP2C9
chr10    95034773    95088997    CYP2C8
BED

# Convert any runs of spaces to a single TAB, remove CRLF if present, and ensure 4 fields
awk -v OFS='\t' '{$1=$1}1' week5_data/cyp2c.hg38.bed | sed 's/\r$//' > week5_data/.tmp.bed
mv week5_data/.tmp.bed week5_data/cyp2c.hg38.bed

# Show the first lines with field separators so we can visually confirm tabs
awk -F'\t' '{printf("F1=%s | F2=%s | F3=%s | F4=%s\n",$1,$2,$3,$4)} NR==3{exit}' week5_data/cyp2c.hg38.bed

# Validate: exactly 4 tab-separated fields per line, numeric coords, start<end
awk -F'\t' 'NF!=4{bad=1}
            {if($2!~/^[0-9]+$/ || $3!~/^[0-9]+$/ || $2>=$3) bad=1}
            END{ if(bad){print "BED validation: FAIL"; exit 1} else {print "BED validation: OK"} }' \
            week5_data/cyp2c.hg38.bed

F1=chr10 | F2=94760662 | F3=94858282 | F4=CYP2C19
F1=chr10 | F2=94936658 | F3=94991091 | F4=CYP2C9
F1=chr10 | F2=95034773 | F3=95088997 | F4=CYP2C8


BED validation: OK


Variant calling with bcftools (one VCF per sample)

In [25]:
%%bash
set -euo pipefail
THREADS=${THREADS:-4}

for SAMPLE in illumina pacbio; do
  BAM=align/${SAMPLE}.chr10.bam
  RAW=vcf/${SAMPLE}.raw.vcf.gz
  OUT=vcf/${SAMPLE}.vcf.gz

  # Quick sanity: contig names should include 'chr10'
  samtools idxstats "$BAM" | head -n1

  bcftools mpileup \
    --threads "$THREADS" \
    -f refs/chr10.fa \
    -Ou \
    -a AD,DP,SP \
    -R week5_data/cyp2c.hg38.bed \
    "$BAM" \
  | bcftools call \
      --threads "$THREADS" \
      -mv -Oz -o "$RAW"

  bcftools norm --threads "$THREADS" -f refs/chr10.fa -m -both -Oz -o "$OUT" "$RAW"
  tabix -p vcf "$OUT"

  echo "Wrote: $OUT"
done

chr10	133797422	307968	1673


[mpileup] 1 samples in 1 input files
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] maximum number of reads per input file set to -d 250
Lines   total/split/joined/realigned/mismatch_removed/dup_removed/skipped:	382/4/0/47/0/0/0


Wrote: vcf/illumina.vcf.gz
chr10	133797422	3126	0


[mpileup] 1 samples in 1 input files
Note: none of --samples-file, --ploidy or --ploidy-file given, assuming all sites are diploid
[mpileup] maximum number of reads per input file set to -d 250
Lines   total/split/joined/realigned/mismatch_removed/dup_removed/skipped:	354/7/0/52/0/0/0


Wrote: vcf/pacbio.vcf.gz


Variant records check

In [29]:
%%bash
set -euo pipefail
echo -n "Illumina variant rows: "
zgrep -vc '^#' vcf/illumina.vcf.gz || true
echo -n "PacBio variant rows:   "
zgrep -vc '^#' vcf/pacbio.vcf.gz || true

Illumina variant rows: 386
PacBio variant rows:   361


Phasing with HapCUT2

In [34]:
%%bash
set -euo pipefail
THREADS=${THREADS:-4}
mkdir -p phase

have() { command -v "$1" >/dev/null 2>&1; }

# Install whatshap if missing (Python package)
if ! have whatshap; then
  pip install --quiet --no-input whatshap
fi

for SAMPLE in illumina pacbio; do
  BAM=align/${SAMPLE}.chr10.bam
  VCF_IN=vcf/${SAMPLE}.vcf.gz
  VCF_OUT=vcf/${SAMPLE}.phased.vcf

  # Read-backed phasing; --ignore-read-groups avoids RG requirements
  # VCF is already chr10-only; adding --chromosome for clarity
  whatshap phase \
    --ignore-read-groups \
    --indels \
    --chromosome chr10 \
    --reference refs/chr10.fa \
    -o "$VCF_OUT" \
    "$VCF_IN" "$BAM"

  bgzip -f "$VCF_OUT"
  tabix -p vcf "${VCF_OUT}.gz"

  echo "Phased VCF: ${VCF_OUT}.gz"
done

This is WhatsHap 2.8 running under Python 3.12.12
Working on 1 sample from 1 family

# Working on contig chr10 in individual align/illumina.chr10.bam
Found 219 usable heterozygous variants (0 skipped due to missing genotypes)
Number of supplementary alignments: 0
Number of non-singleton groups: 545
Skipped 0 groups
Found 7586 reads covering 219 variants
Kept 1171 reads that cover at least two variants each
Selected 617 most phase-informative reads covering 156 variants
Best-case phasing would result in 48 non-singleton phased blocks (0 singletons). 
Phasing 1 sample by solving the MEC problem ...
Largest block contains 14 variants (9.0% of accessible variants) between position 95079296 and 95080679

# Resource usage
Maximum memory usage: 0.319 GB
Time spent reading BAM/CRAM:                    2.7 s
Time spent parsing VCF:                         0.0 s
Time spent selecting reads:                     0.0 s
Time spent phasing:                             0.2 s
Time spent writing VCF:    

Phased VCF: vcf/illumina.phased.vcf.gz


This is WhatsHap 2.8 running under Python 3.12.12
Working on 1 sample from 1 family

# Working on contig chr10 in individual align/pacbio.chr10.bam
Found 225 usable heterozygous variants (0 skipped due to missing genotypes)
Number of supplementary alignments: 0
Number of non-singleton groups: 0
Skipped 0 groups
Found 2831 reads covering 225 variants
Kept 2473 reads that cover at least two variants each
Selected 500 most phase-informative reads covering 225 variants
Best-case phasing would result in 7 non-singleton phased blocks (0 singletons). 
Phasing 1 sample by solving the MEC problem ...
Largest block contains 62 variants (27.6% of accessible variants) between position 95035703 and 95073358

# Resource usage
Maximum memory usage: 0.319 GB
Time spent reading BAM/CRAM:                    0.8 s
Time spent parsing VCF:                         0.0 s
Time spent selecting reads:                     0.1 s
Time spent phasing:                             0.4 s
Time spent writing VCF:        

Phased VCF: vcf/pacbio.phased.vcf.gz


Comparing phased VCFs

In [48]:
%%bash
set -euo pipefail
mkdir -p igv isec

# Make sure isec outputs exist (from your earlier step)
if [ ! -s isec/0002.vcf.gz ]; then
  bcftools isec -p isec -Oz vcf/illumina.phased.vcf.gz vcf/pacbio.phased.vcf.gz
fi

# Helper to extract loci in IGV-friendly "chr:pos-pos"
loci_from_vcf() {
  local vcf="$1" n="$2" vartype="${3:-all}"
  if [ "$vartype" = snp ]; then
    bcftools view -H -v snps "$vcf" | head -n "$n" | awk '{print $1":"$2"-"$2}'
  elif [ "$vartype" = indel ]; then
    bcftools view -H -v indels "$vcf" | head -n "$n" | awk '{print $1":"$2"-"$2}'
  else
    zgrep -v '^#' "$vcf" | head -n "$n" | awk '{print $1":"$2"-"$2}'
  fi
}

# Choose 2 Illumina-only SNVs, 1 PacBio-only INDEL (fallbacks if empty)
ILL_SNPS="$(loci_from_vcf isec/0000.vcf.gz 2 snp || true)"
PB_INDEL="$(loci_from_vcf isec/0001.vcf.gz 1 indel || true)"

# Fallbacks
if [ -z "$ILL_SNPS" ]; then ILL_SNPS="$(loci_from_vcf isec/0000.vcf.gz 2 all || true)"; fi
if [ -z "$PB_INDEL" ]; then PB_INDEL="$(loci_from_vcf isec/0001.vcf.gz 1 all || true)"; fi

# Assemble list (remove blanks/dups)
{
  echo "$ILL_SNPS"
  echo "$PB_INDEL"
} | sed '/^$/d' | awk '!seen[$0]++' > igv/sites.txt

echo "Chosen loci (put these into IGV's search box):"
nl -ba igv/sites.txt

Chosen loci (put these into IGV's search box):
     1	chr10:94772788-94772788
     2	chr10:94772850-94772850
     3	chr10:94779566-94779566


In [49]:
%%bash
set -euo pipefail
[ -s igv/sites.txt ] || { echo "No igv/sites.txt; run the previous cell first."; exit 1; }

{
  echo -e "site\tillumina_GT\tillumina_DP\tillumina_AD\tpacbio_GT\tpacbio_DP\tpacbio_AD"
  while read -r site; do
    ill="$(bcftools query -f '%GT\t%DP\t%AD\n' -r "$site" vcf/illumina.phased.vcf.gz 2>/dev/null || true)"
    [ -z "$ill" ] && ill=".	.	."
    pb="$(bcftools query -f '%GT\t%DP\t%AD\n' -r "$site" vcf/pacbio.phased.vcf.gz 2>/dev/null || true)"
    [ -z "$pb" ] && pb=".	.	."
    echo -e "$site\t$ill\t$pb"
  done < igv/sites.txt
} | tee isec/discordant_summary.tsv

echo
echo "Preview:"
column -t isec/discordant_summary.tsv | sed 's/\t/  /g'

site	illumina_GT	illumina_DP	illumina_AD	pacbio_GT	pacbio_DP	pacbio_AD
chr10:94772788-94772788	.	.	.	.	.	.
chr10:94772850-94772850	.	.	.	.	.	.
chr10:94779566-94779566	.	.	.	.	.	.

Preview:
site                     illumina_GT  illumina_DP  illumina_AD  pacbio_GT  pacbio_DP  pacbio_AD
chr10:94772788-94772788  .            .            .            .          .          .
chr10:94772850-94772850  .            .            .            .          .          .
chr10:94779566-94779566  .            .            .            .          .          .


### IGV screenshots (discordant variants)

Illumina-only 94772788-94772788
![](igv/snaps/illumina_only_94772788-94772788.png)

Illumina-only 94772850-94772850
![](igv/snaps/illumina_only_94772850-94772850.png)

PacBio-illumina 94779566-94779566
![](igv/snaps/pacbio_illumina_94779566-94779566.png)

From these screen shots we can see

From image one 94,772,788:
Illumina: strong, consistent ALT pileup; high apparent MQ; minimal clipping.  
- PacBio: no reads over the base.  
- Conclusion: Likely a true SNV supported by Illumina; PacBio missed it due to local coverage dropout.

From image two 94,772,850:
- Illumina: clean, repeated ALT signal; good coverage; minimal clipping.  
- PacBio: no overlap.  
- Conclusion: Same as above — likely real SNV; PacBio lacks coverage at this locus.

From image three 94,779,566:
- PacBio: consistent indel across many long reads in a poly-T context (purple indel glyphs).  
- Illumina: scattered mismatches/soft-clipping; no clear indel representation.  
- Conclusion: Likely a true indel captured by long reads; short reads struggle in homopolymers.

In [51]:
%%bash
# Create star allele marker file
set -euo pipefail
mkdir -p week5_data

cat > week5_data/star_markers.hg38.tsv <<'EOF'
gene	star	rsid	chr	pos	ref	alt	notes
CYP2C19	*2	rs4244285	chr10	94781859	G	A	LOF splice; defines *2
CYP2C19	*3	rs4986893	chr10	94780653	G	A	LOF stop; defines *3
CYP2C19	*17	rs12248560	chr10	94761900	C	T	Promoter; increased function *17

CYP2C9	*2	rs1799853	chr10	94942290	C	T	R144C; defines *2
CYP2C9	*3	rs1057910	chr10	94942215	A	C	I359L; defines *3

# CYP2C8: *3 is a 2-marker haplotype; both must be on the SAME haplotype
CYP2C8	*2	rs11572103	chr10	95058349	T	A	I269F; defines *2
CYP2C8	*3a	rs11572080	chr10	95067273	G	A	R139K; with rs10509681
CYP2C8	*3b	rs10509681	chr10	95038992	A	G	K399R; with rs11572080
CYP2C8	*4	rs1058930	chr10	95058362	G	C	I264M; defines *4
EOF

echo "Wrote week5_data/star_markers.hg38.tsv"

Wrote week5_data/star_markers.hg38.tsv


In [53]:
# Lightweight star-caller using your phased VCFs 
import subprocess, shlex, csv, pathlib, sys
from collections import defaultdict

VCFS = {
    "illumina": "vcf/illumina.phased.vcf.gz",
    "pacbio"  : "vcf/pacbio.phased.vcf.gz",
}

markers = []
with open("week5_data/star_markers.hg38.tsv") as f:
    for row in csv.DictReader(f, delimiter="\t"):
        if row["gene"].strip() and not row["gene"].startswith("#"):
            row["pos"] = int(row["pos"])
            markers.append(row)

def bcftools_query(vcf, chrom, pos):
    """Return (ref, alts, GT) for this site; defaults to reference 0|0 if absent."""
    fmt = r'%REF\t%ALT\t[%GT]\n'
    cmd = f"bcftools query -r {chrom}:{pos}-{pos} -f '{fmt}' {shlex.quote(vcf)}"
    p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if p.returncode != 0 or not p.stdout.strip():
        return None, None, "0|0"
    ref, alts, gt = p.stdout.strip().split("\t")
    return ref, alts, gt

def alt_on_haps(gt):
    """Return (hap1_is_alt, hap2_is_alt) for biallelic sites."""
    if gt in (".", "./.", ".|."):
        return (False, False)
    # Expect phased: a|b ; if unphased a/b treat as present but unknown phase
    sep = "|" if "|" in gt else "/"
    a,b = gt.split(sep)
    # For biallelic, '1' means ALT, '0' REF. (Ignore multiallelic for these markers.)
    return (a == "1", b == "1")

def call_gene(vcf_path, gene_markers):
    """Return haplotype-level star labels for a single gene."""
    # hap labels start as *1
    hap = ["*1", "*1"]
    notes = []

    # convenience lookups
    def set_star(h, star):
        if hap[h] == "*1":
            hap[h] = star
        elif hap[h] != star:
            notes.append(f"hap{h+1} has {hap[h]} and {star} -> complex")
            hap[h] = hap[h] + "+" + star

    # First pass: single-marker stars
    presence = {}  # key: rsid -> (h1_alt, h2_alt)
    by_rsid   = {m["rsid"]: m for m in gene_markers}
    for m in gene_markers:
        ref, alts, gt = bcftools_query(vcf_path, m["chr"], m["pos"])
        h1, h2 = alt_on_haps(gt)
        presence[m["rsid"]] = (h1, h2)

        # apply simple stars immediately
        if m["gene"] == "CYP2C19" and m["star"] in ("*2","*3","*17"):
            if h1: set_star(0, m["star"])
            if h2: set_star(1, m["star"])

        if m["gene"] == "CYP2C9" and m["star"] in ("*2","*3"):
            if h1: set_star(0, m["star"])
            if h2: set_star(1, m["star"])

        if m["gene"] == "CYP2C8" and m["star"] in ("*2","*4"):
            if h1: set_star(0, m["star"])
            if h2: set_star(1, m["star"])

    # Second pass: CYP2C8 *3 requires BOTH rs11572080 and rs10509681 on the SAME haplotype
    if gene_markers[0]["gene"] == "CYP2C8":
        h1_a, h2_a = presence.get("rs11572080", (False, False))  # *3a
        h1_b, h2_b = presence.get("rs10509681", (False, False))  # *3b
        if h1_a and h1_b: set_star(0, "*3")
        if h2_a and h2_b: set_star(1, "*3")
        # If split across haps, mention it
        if (h1_a and h2_b) or (h2_a and h1_b):
            notes.append("CYP2C8 *3 markers seen on opposite haplotypes -> not *3")

    diplotype = "/".join(hap)
    return diplotype, notes, presence

# Run for both VCFs
by_gene = defaultdict(list)
for tech, vcf in VCFS.items():
    # group markers by gene
    genes = defaultdict(list)
    for m in markers: genes[m["gene"]].append(m)
    for gene, gms in genes.items():
        dip, notes, presence = call_gene(vcf, gms)
        by_gene[gene].append((tech, dip, notes))

# Pretty print summary
print("### Star-allele diplotype calls (from phased VCFs)")
for gene, rows in sorted(by_gene.items()):
    print(f"\n{gene}:")
    for tech, dip, notes in rows:
        print(f"  - {tech:8s}: {dip}")
        for n in notes:
            print(f"      note: {n}")


### Star-allele diplotype calls (from phased VCFs)

CYP2C19:
  - illumina: *1/*17
  - pacbio  : *1/*17

CYP2C8:
  - illumina: *1/*2
  - pacbio  : *1/*2

CYP2C9:
  - illumina: *1/*1
  - pacbio  : *1/*1


### Star-allele calls (from phased VCFs)

**CYP2C19 → *1/*17 (Rapid metabolizer)**
- Because: heterozygous for promoter variant **rs12248560 C>T** (defines *17).  
- And: loss-of-function markers **rs4244285 (*2)** and **rs4986893 (*3)** are **absent** (REF on both haps).  
- Phasing: ALT is present on exactly one haplotype in both Illumina and PacBio phased VCFs → *1/*17.

**CYP2C8 → *1/*2 (decreased function allele present)**
- Because: heterozygous for **rs11572103 T>A** (defines *2).  
- And: *3 requires **both** rs11572080 and rs10509681 on the **same** haplotype — not observed; *4 (rs1058930) also absent.  
- Phasing: ALT on one haplotype only → *1/*2.

**CYP2C9 → *1/*1 (normal metabolizer)**
- Because: common decreased-function alleles **rs1799853 (*2)** and **rs1057910 (*3)** are **absent** (REF/REF).

**Notes.** These calls use common PharmVar/ClinPGx markers on hg38 chr10 and the **phased** VCFs so, can tell which haplotype each ALT is on. I read some star systems also consider rare variants/CNVs but I did not assess those here.